In [32]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

import pandas as pd
import numpy as np
import torch, torch.nn as nn
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from data.data_manager import *

tree_path = 'decision_tree.pkl'

# Load data
data_path = 'IP_decisions_2/IP_decisions_noise=0.01_model=1200'
data = pd.read_csv(data_path+'_policy.csv')

print(data.head())

X = data.drop(columns=['action'])
y = data['action']

# Load the tree
import pickle
pickle_in = open(tree_path, "rb")

tree = pickle.loads(pickle_in.read())

def model_predict_fn(state_vector, model_path):
    # use cuda if available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # load the model
    model_demo = nn.Sequential(
        nn.Linear(8, 200),
        nn.SiLU(),
        nn.Linear(200, 200),
        nn.SiLU(),
        nn.Linear(200, 200),
        nn.SiLU(),
        nn.Linear(200, 1),
    )
    model_demo.load_state_dict(torch.load(model_path))
    model_demo.eval()
    prediction = model_demo(torch.tensor(state_vector.astype(np.float32))).detach().numpy()
    return prediction

    time  Site Outdoor Air Drybulb Temperature(Environment)  \
0   9.50                                           4.518009   
1  11.75                                           6.158388   
2  15.75                                           0.710889   
3   8.25                                          12.752567   
4  16.75                                          15.004332   

   Site Outdoor Air Relative Humidity(Environment)  \
0                                        70.059378   
1                                        48.825850   
2                                        92.624592   
3                                        66.065636   
4                                        75.874350   

   Site Wind Speed(Environment)  \
0                      8.413355   
1                      4.128619   
2                      6.540552   
3                      5.431762   
4                      3.251662   

   Site Direct Solar Radiation Rate per Area(Environment)  \
0                       

In [33]:
from datetime import datetime

def probabilistic_verify(noise_level, samples, tree, model_path, save_path='verification_result.csv'):

    Historical_X = pd.read_csv('data/X_pittsburgh.csv')

    for sample_idx in range(samples):
        if sample_idx % 200 == 0:
            print('Sample #{}'.format(sample_idx))

        df = []

        # sample a number from 0 to size of Historical_X - 21 that mod 96 > 31 and mod 96 < 84
        current_step = np.random.randint(0, Historical_X.shape[0]-21)
        while current_step % 96 < 31 or current_step % 96 > 84:
            current_step = np.random.randint(0, Historical_X.shape[0]-21)
        # get the row of Historical_X at index current_step
        input_x = Historical_X.iloc[current_step]

        zone_temp = input_x['Zone Air Temperature(SPACE1-1)']

        while zone_temp < 20.0 or zone_temp > 23.5 or current_step % 96 < 31 or current_step % 96 > 84:
            current_step = np.random.randint(0, Historical_X.shape[0]-21)
            while current_step % 96 < 31 or current_step % 96 > 84:
                current_step = np.random.randint(0, Historical_X.shape[0]-21)
            # get the row of Historical_X at index current_step
            input_x = Historical_X.iloc[current_step]
            zone_temp = input_x['Zone Air Temperature(SPACE1-1)']

        people_count = input_x['Zone People Occupant Count(SPACE1-1)']

        noised_input = input_x + np.random.normal(0, noise_level*np.std(input_x, axis=0), input_x.shape)
        # separate obs and disturbance
        obs = noised_input['Zone Air Temperature(SPACE1-1)']
        disturbance = noised_input.drop('Zone Air Temperature(SPACE1-1)')
        disturbance = disturbance.drop('Zone Thermostat Heating Setpoint Temperature(SPACE1-1)')
        disturbance = disturbance.drop('Zone Thermostat Cooling Setpoint Temperature(SPACE1-1)')

        '''
        For any of the 'Site Outdoor Air Relative Humidity(Environment)',
       'Site Wind Speed(Environment)',
       'Site Direct Solar Radiation Rate per Area(Environment)',
        if the value is negative, set it to 0
        '''
        disturbance['Site Outdoor Air Relative Humidity(Environment)'] = max(0, disturbance['Site Outdoor Air Relative Humidity(Environment)'])
        disturbance['Site Wind Speed(Environment)'] = max(0, disturbance['Site Wind Speed(Environment)'])
        disturbance['Site Direct Solar Radiation Rate per Area(Environment)'] = max(0, disturbance['Site Direct Solar Radiation Rate per Area(Environment)'])
        
        # set 'Zone People Occupant Count(SPACE1-1)' to people_count
        disturbance['Zone People Occupant Count(SPACE1-1)'] = people_count

        input_vars = ['time',
                  'Site Outdoor Air Drybulb Temperature(Environment)',
                  'Site Outdoor Air Relative Humidity(Environment)',
                  'Site Wind Speed(Environment)',
                  'Site Direct Solar Radiation Rate per Area(Environment)',
                  'Zone People Occupant Count(SPACE1-1)',]
    

        obs_in = [current_step % 96 / 4]
        for col in input_vars[1:]:
            obs_in.append(disturbance[col])
        obs_in.append(zone_temp)

        action = tree.predict([obs_in])[0]

        new_action_mapping = {
            0: (15, 30),
            1: (16, 29),
            2: (17, 28),
            3: (18, 27),
            4: (19, 26),
            5: (20, 25),
            6: (21, 24),
            7: (22, 23),
            8: (22, 22),
            9: (21, 21)
        }

        for i in new_action_mapping.keys():
            if new_action_mapping[i][0] == int(action):
                action = i
                break

        environment_var = ['Site Outdoor Air Drybulb Temperature(Environment)', 
                           'Site Outdoor Air Relative Humidity(Environment)', 
                           'Site Wind Speed(Environment)', 
                           'Site Direct Solar Radiation Rate per Area(Environment)', 
                           'Zone People Occupant Count(SPACE1-1)']

        environment_forecast_vector = np.array([disturbance[var] for var in environment_var]).reshape(1, -1)
        zone_temperature_vector = np.array([zone_temp]).reshape(1, -1)
        heating_setpoint = new_action_mapping[action][0]
        cooling_setpoint = new_action_mapping[action][1]
        action_vector = np.array([heating_setpoint, cooling_setpoint]).reshape(1, -1)

        state_vector = np.concatenate([environment_forecast_vector, zone_temperature_vector, action_vector], axis=1)

        next_zone_temperature = model_predict_fn(state_vector, model_path)

        if (next_zone_temperature < 23.5) and (next_zone_temperature > 20.0):
            verified = 1
        else:
            verified = 0

        df.append([current_step % 96 / 4, 
                disturbance['Site Outdoor Air Drybulb Temperature(Environment)'],
                disturbance['Site Outdoor Air Relative Humidity(Environment)'],
                disturbance['Site Wind Speed(Environment)'],
                disturbance['Site Direct Solar Radiation Rate per Area(Environment)'],
                disturbance['Zone People Occupant Count(SPACE1-1)'],
                obs,
                verified])

        # a csv file already exists at the save_path, append to it
        if os.path.exists(save_path):
            df_csv = pd.DataFrame(df, 
                            columns=['time',
                            'Site Outdoor Air Drybulb Temperature(Environment)',
                            'Site Outdoor Air Relative Humidity(Environment)',
                            'Site Wind Speed(Environment)',
                            'Site Direct Solar Radiation Rate per Area(Environment)',
                            'Zone People Occupant Count(SPACE1-1)',
                            'Zone Air Temperature(SPACE1-1)', 
                            'verified'])
            df_csv.to_csv(save_path, mode='a', header=False)
        # a csv file does not exist at the save_path, create it
        else:
            df_csv = pd.DataFrame(df, 
                                columns=['time',
                                'Site Outdoor Air Drybulb Temperature(Environment)',
                                'Site Outdoor Air Relative Humidity(Environment)',
                                'Site Wind Speed(Environment)',
                                'Site Direct Solar Radiation Rate per Area(Environment)',
                                'Zone People Occupant Count(SPACE1-1)',
                                'Zone Air Temperature(SPACE1-1)', 
                                'verified'])
            df_csv.to_csv(save_path)

In [34]:
probabilistic_verify(0.01, 1000, tree, 'zmodels/model_pittsburgh_winter_1200.pth', save_path='verification_result_pittsburgh.csv')

Sample #0
Sample #200
Sample #400
Sample #600
Sample #800


In [35]:
# Load the verification result
df = pd.read_csv('verification_result_pittsburgh.csv')

# count the number of verified samples
verified = df[df['verified'] == 1]
print('Verified samples: {}'.format(verified.shape[0]))

Verified samples: 946
